# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0b34d5b310>
├── 'a' --> tensor([[-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840]])
└── 'x' --> <FastTreeValue 0x7f0b34d5b910>
    └── 'c' --> tensor([[ 1.2058,  1.2438,  0.5276, -0.1902],
                        [ 0.5533,  1.0221,  1.1639,  0.5424],
                        [ 0.9086, -0.1583, -1.1895,  0.5367]])

In [4]:
t.a

tensor([[-0.7735,  1.0406,  0.5373],
        [-2.6181, -0.1628, -0.0840]])

In [5]:
%timeit t.a

63.9 ns ± 0.0359 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0b34d5b310>
├── 'a' --> tensor([[-0.8802, -0.8358,  1.3796],
│                   [-0.9178,  1.0205, -0.1850]])
└── 'x' --> <FastTreeValue 0x7f0b34d5b910>
    └── 'c' --> tensor([[ 1.2058,  1.2438,  0.5276, -0.1902],
                        [ 0.5533,  1.0221,  1.1639,  0.5424],
                        [ 0.9086, -0.1583, -1.1895,  0.5367]])

In [7]:
%timeit t.a = new_value

66.4 ns ± 0.0439 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7735,  1.0406,  0.5373],
               [-2.6181, -0.1628, -0.0840]]),
    x: Batch(
           c: tensor([[ 1.2058,  1.2438,  0.5276, -0.1902],
                      [ 0.5533,  1.0221,  1.1639,  0.5424],
                      [ 0.9086, -0.1583, -1.1895,  0.5367]]),
       ),
)

In [10]:
b.a

tensor([[-0.7735,  1.0406,  0.5373],
        [-2.6181, -0.1628, -0.0840]])

In [11]:
%timeit b.a

59 ns ± 0.0367 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4872,  0.8893, -0.6562],
               [ 1.6986,  1.7166,  0.0146]]),
    x: Batch(
           c: tensor([[ 1.2058,  1.2438,  0.5276, -0.1902],
                      [ 0.5533,  1.0221,  1.1639,  0.5424],
                      [ 0.9086, -0.1583, -1.1895,  0.5367]]),
       ),
)

In [13]:
%timeit b.a = new_value

498 ns ± 0.274 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

820 ns ± 0.764 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

149 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

149 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f0b0833c8b0>
├── 'a' --> tensor([[[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]],
│           
│                   [[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]],
│           
│                   [[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]],
│           
│                   [[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]],
│           
│                   [[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]],
│           
│                   [[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]],
│           
│                   [[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]],
│           
│                   [[-0.7735,  1.0406,  0.5373],
│                    [-2.6181, -0.1628, -0.0840]]])
└── 'x' --> <FastTreeValue 0x7f0b0833c400>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0a7d0bfdc0>
├── 'a' --> tensor([[-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840],
│                   [-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840],
│                   [-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840],
│                   [-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840],
│                   [-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840],
│                   [-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840],
│                   [-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840],
│                   [-0.7735,  1.0406,  0.5373],
│                   [-2.6181, -0.1628, -0.0840]])
└── 'x' --> <FastTreeValue 0x7f0b10274520>
    └── 'c' --> tensor([[ 1.2058,  1.2438,  0.5276, -0.1902],
                        [ 0.5533,  1.0221,  1.1639,  0.5424],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 46.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.5 µs ± 100 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2058,  1.2438,  0.5276, -0.1902],
                       [ 0.5533,  1.0221,  1.1639,  0.5424],
                       [ 0.9086, -0.1583, -1.1895,  0.5367]],
              
                      [[ 1.2058,  1.2438,  0.5276, -0.1902],
                       [ 0.5533,  1.0221,  1.1639,  0.5424],
                       [ 0.9086, -0.1583, -1.1895,  0.5367]],
              
                      [[ 1.2058,  1.2438,  0.5276, -0.1902],
                       [ 0.5533,  1.0221,  1.1639,  0.5424],
                       [ 0.9086, -0.1583, -1.1895,  0.5367]],
              
                      [[ 1.2058,  1.2438,  0.5276, -0.1902],
                       [ 0.5533,  1.0221,  1.1639,  0.5424],
                       [ 0.9086, -0.1583, -1.1895,  0.5367]],
              
                      [[ 1.2058,  1.2438,  0.5276, -0.1902],
                       [ 0.5533,  1.0221,  1.1639,  0.5424],
                       [ 0.9086, -0.1583, -1.1895,  0.5367]],

In [26]:
%timeit Batch.stack(batches)

77.5 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2058,  1.2438,  0.5276, -0.1902],
                      [ 0.5533,  1.0221,  1.1639,  0.5424],
                      [ 0.9086, -0.1583, -1.1895,  0.5367],
                      [ 1.2058,  1.2438,  0.5276, -0.1902],
                      [ 0.5533,  1.0221,  1.1639,  0.5424],
                      [ 0.9086, -0.1583, -1.1895,  0.5367],
                      [ 1.2058,  1.2438,  0.5276, -0.1902],
                      [ 0.5533,  1.0221,  1.1639,  0.5424],
                      [ 0.9086, -0.1583, -1.1895,  0.5367],
                      [ 1.2058,  1.2438,  0.5276, -0.1902],
                      [ 0.5533,  1.0221,  1.1639,  0.5424],
                      [ 0.9086, -0.1583, -1.1895,  0.5367],
                      [ 1.2058,  1.2438,  0.5276, -0.1902],
                      [ 0.5533,  1.0221,  1.1639,  0.5424],
                      [ 0.9086, -0.1583, -1.1895,  0.5367],
                      [ 1.2058,  1.2438,  0.5276, -0.1902],
                   

In [28]:
%timeit Batch.cat(batches)

147 µs ± 622 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

326 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
